> # Traitement Automatique de texte en IA projet
> 
> > Antoine Vidal-Mazuy - Yann Brault
> > 28/12/2021 Université Côtes d'azur

<br>

> ## Introduction
>
> > blablzqdbd bzqjkdb qz

<br>
<br>

> ## Dataset utilisés
>
> > #### Datasets de base:



> ## Base line:
>
> > #### Prédicteurs de base:
> > > Pour commencer notre projet, nous avons fait deux systèmes de prédictions simples. <br>
> > > De base, une classe est assignée à chaque avis. Si la note attribuée est supérieure ou égale à 12 alors on assigne l'avis à la classe 1 comme signe de bon jeu. <br>
> > > En revanche, si la note est strictement inférieure à 12 alors, l'avis est catégorisé par la classe 0, celle des mauvais jeux. <br>
> > > ##### Prédicteur par comptage de mots:
> > > L'idée ici est plutôt naïve. Si le commentaire contient plus de mots au sens négatifs, alors on prédit le jeu comme étant mauvais. À l'inverse, <br>
> > > si la condition énoncée n'est pas validée alors le jeu est considéré comme étant bon. <br>
> > > Le code marche comme ceci:


In [12]:
from tqdm import trange
import pandas as pd

def predict(data: pd.DataFrame) -> tuple[list[int], list[int]]:
    classes_predicted = []
    classes_base = []

    for i in trange(data.shape[0]):
        row = data.iloc[i]
        neg = row['negative_words']
        pos = row['positive_words']
        
        predict = 1
        if neg > pos: 
            predict = 0
        
        base = row['classe_bon_mauvais']
        classes_base.append(int(base))
        classes_predicted.append(int(predict))
    
    return classes_base, classes_predicted
    
def compute_confusion_matrix(classes_base, classes_predicted):
    
    M = [[0, 0], [0, 0]]

    for i in range(len(classes_base)):
        M[classes_base[i]][classes_predicted[i]] += 1
    
    return M

def accuracy(TN, TP, FN, FP):
    size_list = len(TN)
    accuracy_sum = 0
    for i in range(size_list):
        accuracy_sum += (TP[i] + TN[i]) / (TP[i] + TN[i] + FN[i] + FP[i])
    return accuracy_sum / size_list

def recall(TP, FN):
    size_list = len(TP)
    recall_sum = 0
    for i in range(size_list):
        recall_sum += TP[i] / (TP[i] + FN[i])
    return recall_sum / size_list

if __name__ == "__main__":
    data = pd.read_csv('dataset/csv/base_predictor.csv')
    classes_base, classes_predicted = predict(data)
    M = compute_confusion_matrix(classes_base, classes_predicted)
    print(M)
    TP = [0,0]
    TN = [0,0]
    FP = [0,0]
    FN = [0,0]
    Total = M[0][0] + M[0][1] + M[1][0] + M[1][1]

    for i in range(2):
        TP[i] = M[i][i]
        for j in range(2):
            FN[i] += M[i][j]
            FP[i] += M[j][i]
        
        FN[i] -= M[i][i]
        FP[i] -= M[i][i]
        TN[i] = Total - FP[i] - FN[i] + TP[i]

    for i in range(2):
        print(f"cat {i}: TP:{TP[i]}, TN:{TN[i]}, FP:{FP[i]}, FN:{FN[i]}\n")

    print (f"accuracy: {accuracy(TN, TP, FN, FP)}\n")
    print (f"accuracy: {recall(TP, FN)}\n")


100%|██████████| 102478/102478 [00:06<00:00, 16080.03it/s]

[[15110, 4847], [48369, 34152]]
cat 0: TP:15110, TN:64372, FP:48369, FN:4847

cat 1: TP:34152, TN:83414, FP:4847, FN:48369

accuracy: 0.64368358122073

accuracy: 0.585493057720152



> > > ##### Prédicteur toujours bon:
> > > L'idée ici est plus que simple. Peu importe que la classe de base soit bonne ou mauvaise, on prédit toujours que le jeu est bon. <br>

In [13]:
def predict_class(data): #here we will assign a class number according to the rate. 1 for good games and 2 for bad games.
    classes_predicted = []
    classes_base = []
    predict = 1
    for i in range(data.shape[0]):
        row = data.iloc[i]
        base = row['classe_bon_mauvais']
        classes_base.append(int(base))
        classes_predicted.append(int(predict))
    return classes_base, classes_predicted

def accuracy(TN, TP, FN, FP):
    size_list = len(TN)
    accuracy_sum = 0
    for i in range(size_list):
        accuracy_sum += (TP[i] + TN[i]) / (TP[i] + TN[i] + FN[i] + FP[i])
    return accuracy_sum / size_list

def recall(TP, FN):
    size_list = len(TP)
    recall_sum = 0
    for i in range(size_list):
        recall_sum += TP[i] / (TP[i] + FN[i])
    return recall_sum / size_list

def CM(classes_base, classes_predicted):
    M = [[0, 0], [0, 0]]

    for i in range(len(classes_base)):
        M[classes_base[i]][classes_predicted[i]] += 1

    TP = [0,0]
    TN = [0,0]
    FP = [0,0]
    FN = [0,0]
    Total = M[0][0] + M[0][1] + M[1][0] + M[1][1]

    for i in range(2):
        TP[i] = M[i][i]
        for j in range(2):
            FN[i] += M[i][j]
            FP[i] += M[j][i]
        
        FN[i] -= M[i][i]
        FP[i] -= M[i][i]
        TN[i] = Total - FP[i] - FN[i] + TP[i]

    for i in range(2):
        print(f"cat {i}: TP:{TP[i]}, TN:{TN[i]}, FP:{FP[i]}, FN:{FN[i]}\n")

    accu = accuracy(TN, TP, FN, FP)
    rec = recall(TP, FN)

    print(f"the globla accuracy is {accu:.2f}\n")
    print(f"the globla recall is {rec:.2f}\n")

if __name__ == '__main__': 
    my_data = pd.read_csv("./dataset/csv/base_predictor.csv")
    classes_base, classes_predicted = predict_class(my_data)
    CM(classes_base, classes_predicted)

cat 0: TP:0, TN:82521, FP:0, FN:19957

cat 1: TP:82521, TN:165042, FP:19957, FN:0

the globla accuracy is 0.87

the globla recall is 0.50

